In [ ]:
import matplotlib.pyplot as plt
import os
import random 
import time
import re
from tqdm import tqdm

from src.problem import MultiODProblem
from src.solution import MultiODSolution
from src.utils import read_instance_data
from src.gurobi.gurobi_formulation import formulation
from src.gurobi.utils import display_gurobi_result, generate_paths_from_gurobi_result

In [ ]:
num_O = '005'
resample_k = 5

In [ ]:
def run_experiment(instance):
    report_str = ""
    locations = read_instance_data(instance)
    p = MultiODProblem(locations=locations, ignore_to_dummy_cost=False, ignore_from_dummy_cost=False)
    p.convert_distance_matrix_to_int()
    start_time = time.time()
    X, m = formulation(p, '2D')
    m.setParam('TimeLimit', 60*60*2)
    m.optimize()
    end_time = time.time()
    # fig = display_gurobi_result(X, m, p)
    # plt.show()
    paths = generate_paths_from_gurobi_result(X)
    report_str += f"{paths}\n"
    report_str += f"The solution is feasible: {p.is_feasible(paths)}\n"
    report_str += f"The solution cost is: {m.objVal}\n"
    report_str += f"Execution time: {end_time-start_time} seconds\n"
    return report_str, m.objVal

In [ ]:
instance_dir = os.path.join('data', 'tsppdlib', 'instances', 'random-uniform')
instances = [i for i in os.listdir(instance_dir) if i.endswith('.tsp')]

sub_instances = [i for i in instances if '-' + num_O + '-' in i]
sub_instances = random.sample(sub_instances, k=resample_k)

In [ ]:
selected_files = sub_instances
for selected_file in tqdm(selected_files, desc="Processing files", unit="file"):
    instance = os.path.join(instance_dir, selected_file)
    results = run_experiment(instance)
    file_name = selected_file.replace(".tsp", "")
    file_path = f"../tmp/gurobi/{file_name}.{int(results[1])}.txt"
    with open(file_path, 'w+') as file:
        file.write(f"The result for {selected_file}\n")
        file.write(f"{results[0]}\n")